<a href="https://colab.research.google.com/github/rafaelinfopiaui/analise-sentimentos-cpweekend/blob/main/notebooks/Desenvolvimento_Analise_Sentimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Análise de Sentimentos para a Campus Party Weekend Piauí 2025

**Objetivo:** Desenvolver um protótipo de Machine Learning capaz de classificar o sentimento de textos em português (positivo, negativo, neutro) a partir de dados coletados de redes sociais.

**Projeto de Extensão:** Engenharia de Computação com IA - UNI-CET

**Equipe de Desenvolvedores:**
* Rafael Oliveira
* Ailton Medeiros
* Lais Eulálio
* Antônio Wilker
* Isaac Aragão
* Paula Iranda

**Docente Orientador:** Prof. Dr. Artur Felipe da Silva Veloso

**Etapas deste Notebook:**
1.  **Configuração do Ambiente:** Conexão com o Google Drive e importação de bibliotecas.
2.  **Carga e Análise dos Dados:** Carregar o dataset e fazer uma análise exploratória inicial.
3.  **Pré-Processamento:** Limpeza e preparação dos textos para o modelo.
4.  **Treinamento do Modelo:** Construção, treino e avaliação de um modelo de classificação.
5.  **Salvamento do Modelo:** Exportar o modelo treinado para ser usado na aplicação com Streamlit.

## 1. Configuração do Ambiente

Nesta primeira etapa, a equipe irá conectar o Colab ao Google Drive para acessar os dados, importar todas as bibliotecas que serão utilizadas no projeto e definir os caminhos dos arquivos para manter o código organizado e acessível a todos.

In [2]:
## 1. Configuração do Ambiente

# 1.1 - INSTALAÇÃO E UPGRADE DE PACOTES (CRUCIAL PARA CORREÇÃO)
!pip install pandas scikit-learn streamlit nltk joblib
!pip install numpy --upgrade
!pip install scikit-learn --upgrade
!pip install joblib --upgrade
print("Pacotes de ML instalados/atualizados com sucesso!")

# 1.2 - Conectar ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 1.3 - LIMPEZA DE CACHE DO AMBIENTE (FINAL)
# Movemos para o início para que pacotes sejam carregados corretamente
%reset -f
print("\nCache de pacotes limpo. Recarregando ambiente...")

# 1.4 - REDEFINIR Variáveis e Mudar diretório (APÓS O RESET)
# A variável PASTA_PROJETO precisa ser redefinida aqui
PASTA_PROJETO = '/content/drive/MyDrive/Projeto_CampusParty_Sentimentos'
%cd {PASTA_PROJETO}

# 1.5 - Importar bibliotecas essenciais (TUDO SERÁ IMPORTADO A PARTIR DAQUI)
import pandas as pd
import numpy as np
import re
import nltk
import joblib

# 1.6 - Importar módulos do Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# 1.7 - Definir Caminhos (USANDO A PASTA_PROJETO CORRIGIDA)
CAMINHO_DADOS_BRUTOS = f'{PASTA_PROJETO}/data/dataset_bruto.csv'
CAMINHO_DADOS_LIMPOS = f'{PASTA_PROJETO}/data/dados_limpos.csv'
CAMINHO_MODELO = f'{PASTA_PROJETO}/saved_models/modelo_sentimento.joblib'

print("\nAmbiente configurado e pacotes prontos!")

  Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (9.5 MB)
  Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (9.5 MB)
Pacotes de ML instalados/atualizados com sucesso!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Cache de pacotes limpo. Recarregando ambiente...
/content/drive/MyDrive/Projeto_CampusParty_Sentimentos

Ambiente configurado e pacotes prontos!


## 2. Carga e Análise Exploratória dos Dados (EDA)

Nesta etapa, a equipe carrega o dataset e realiza uma verificação rápida para entender sua estrutura, a quantidade de dados e a presença de valores nulos.

**Nota sobre o Fluxo de Trabalho:** Para garantir a estabilidade e contornar um problema persistente de sincronização com o Google Drive, o carregamento dos dados será feito através do **upload direto** para a sessão do Colab a cada vez que o notebook for iniciado. O código a seguir já está preparado para ler o arquivo deste ambiente local temporário, utilizando os parâmetros de leitura que se provaram eficazes durante os testes.

In [3]:
## 2. Carga e Análise Exploratória dos Dados (EDA)

# 2.1 - Carregar o dataset
try:
    df = pd.read_csv(CAMINHO_DADOS_BRUTOS)
    print(f"Dataset carregado com sucesso! Total de linhas: {len(df)}")
except FileNotFoundError:
    print(f"Erro: Arquivo não encontrado em '{CAMINHO_DADOS_BRUTOS}'. Verifique o caminho e o nome do arquivo.")

# 2.2 - Mapear colunas do dataset Reddit para o padrão do nosso projeto
# O corpo (texto) do comentário.
df['texto'] = df['body']
# O score (votos) indica sentimento: > 1 é Positivo (1); <= 1 é Negativo/Neutro (0).
df['sentimento'] = df['score'].apply(lambda x: 1 if x > 1 else 0)

# 2.3 - Visualizar as primeiras linhas
print("\nPrimeiras 5 linhas do dataset (após mapeamento):")
display(df[['body', 'score', 'texto', 'sentimento']].head())

# 2.4 - Verificar informações gerais
print("\nInformações do DataFrame:")
df.info()

# 2.5 - Verificar a distribuição das classes de sentimento
print("\nDistribuição inicial dos sentimentos (0=Negativo/Neutro, 1=Positivo):")
print(df['sentimento'].value_counts(normalize=True))

Dataset carregado com sucesso! Total de linhas: 872153

Primeiras 5 linhas do dataset (após mapeamento):


,body,score,texto,sentimento
0,Limpa os cacos com um pano úmido e borrifa àgu...,1,Limpa os cacos com um pano úmido e borrifa àgu...,0
1,Eu autorizo Xandão,1,Eu autorizo Xandão,0
2,Tem tanta coisa esquisita acontecendo nessa ci...,1,Tem tanta coisa esquisita acontecendo nessa ci...,0
3,"Não é caso de educação, caráter você nasce co...",1,"Não é caso de educação, caráter você nasce co...",0
4,Flan é pudim,1,Flan é pudim,0



Informações do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 872153 entries, 0 to 872152
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   author         872153 non-null  object
 1   body           872151 non-null  object
 2   created_utc    872153 non-null  object
 3   distinguished  8641 non-null    object
 4   edited         872153 non-null  object
 5   comment_id     872153 non-null  object
 6   is_submitter   872153 non-null  bool  
 7   parent_id      872153 non-null  object
 8   score          872153 non-null  int64 
 9   stickied       872153 non-null  bool  
 10  thread_id      872153 non-null  object
 11  version        872153 non-null  int64 
 12  texto          872151 non-null  object
 13  sentimento     872153 non-null  int64 
dtypes: bool(2), int64(3), object(9)
memory usage: 81.5+ MB

Distribuição inicial dos sentimentos (0=Negativo/Neutro, 1=Positivo):
sentimento
1    0.641596
0 

## 3. Limpeza Inicial do DataFrame

Com os dados brutos carregados no DataFrame `df`, o primeiro passo é uma "faxina" estrutural para tornar o dataset mais limpo e fácil de trabalhar. As seguintes ações serão tomadas:

1.  **Limpeza dos Nomes das Colunas:** As aspas extras (`"`) presentes nos nomes das colunas serão removidas.
2.  **Remoção de Linhas Vazias:** As linhas que não contêm comentários (valores nulos na coluna `body`) serão eliminadas, pois são inúteis para a nossa análise de sentimentos.
3.  **Seleção de Colunas Relevantes:** Um novo DataFrame, `df_final`, será criado contendo apenas as colunas mais importantes para o nosso estudo (`author`, `body`, `score`, `created_utc`).

In [4]:
## 3. Limpeza Inicial do DataFrame

# 3.1 - Remoção de linhas com texto vazio ou nulo nas colunas chave
linhas_antes = len(df)
df.dropna(subset=['texto', 'sentimento'], inplace=True)
df = df[df['texto'].str.strip() != ''] # Remove linhas onde o texto está vazio ou só contém espaços

linhas_depois = len(df)
print(f"Linhas removidas devido a valores nulos ou vazios: {linhas_antes - linhas_depois}")
print(f"Total de amostras válidas para o pré-processamento: {linhas_depois}")

Linhas removidas devido a valores nulos ou vazios: 2
Total de amostras válidas para o pré-processamento: 872151


## 4. Pré-processamento Profundo do Texto

Com o DataFrame estruturalmente limpo, o próximo passo é processar o conteúdo da coluna `body`. Nesta etapa, vamos aplicar uma função de limpeza em cada comentário para:

- Padronizar o texto (converter para letras minúsculas).
- Remover "ruídos" como URLs, menções de usuários e caracteres especiais.
- Remover palavras comuns que não carregam sentimento (stopwords como 'o', 'a', 'de', 'que').

O resultado será salvo em uma nova coluna, `body_limpo`, que servirá de base para o nosso modelo de Machine Learning.

In [5]:
## 4. Pré-processamento Profundo do Texto

# 4.1 - Definir a função de limpeza de texto
def limpar_texto(texto):
    if not isinstance(texto, str):
        return ""

    texto = texto.lower()
    texto = re.sub(r'https?://\S+|www\.\S+', '', texto) # Remover URLs
    texto = re.sub(r'@\w+|#\w+', '', texto)             # Remover menções e hashtags
    texto = re.sub(r'[^a-z\s]', '', texto)              # Remover caracteres não-alfabéticos

    tokens = nltk.word_tokenize(texto)
    stopwords_pt = nltk.corpus.stopwords.words('portuguese')
    tokens_limpos = [palavra for palavra in tokens if palavra not in stopwords_pt]

    return ' '.join(tokens_limpos)

# 4.2 - Aplicar a função de limpeza na coluna de texto
print("[Passo 1/2] Aplicando limpeza profunda do texto...")
df['texto_limpo'] = df['texto'].apply(limpar_texto)

# 4.3 - SALVAMENTO DO DATASET LIMPO (Correção de salvamento)
print("\n[Passo 2/2] Salvando o dataset limpo para uso futuro...")
try:
    # Salva apenas o texto limpo e a classificação de sentimento
    df[['texto_limpo', 'sentimento']].to_csv(CAMINHO_DADOS_LIMPOS, index=False)
    print(f"✅ Dataset limpo salvo com sucesso em: {CAMINHO_DADOS_LIMPOS}")
except Exception as e:
    print(f"ERRO ao salvar dados limpos: {e}")

[Passo 1/2] Aplicando limpeza profunda do texto...

[Passo 2/2] Salvando o dataset limpo para uso futuro...
✅ Dataset limpo salvo com sucesso em: /content/drive/MyDrive/Projeto_CampusParty_Sentimentos/data/dados_limpos.csv


## 5. Construção, Treinamento e Avaliação do Modelo

Esta é a etapa central do projeto. Com os dados limpos e pré-processados no DataFrame `df_final`, vamos:

1.  **Criar a Variável Alvo (`y`):** Como nosso dataset não possui uma coluna de sentimento, usaremos a coluna `score` (a pontuação do comentário) como um substituto (*proxy*). Comentários com score alto (>1) serão considerados "positivos", e os demais, "negativos/neutros".
2.  **Dividir os Dados:** Separar o dataset em um conjunto de **treino** (para o modelo aprender) e um conjunto de **teste** (para avaliarmos o quão bem ele aprendeu).
3.  **Construir um `Pipeline`:** Criar uma esteira de produção que primeiro transforma o texto limpo em vetores numéricos (`TfidfVectorizer`) e depois os usa para treinar um modelo de classificação (`LogisticRegression`).
4.  **Treinar e Avaliar:** Executar o treinamento e verificar a performance do modelo com métricas como a acurácia.

In [8]:
## 5. Construção, Treinamento e Avaliação do Modelo

print("\n--- Iniciando a Construção e Treinamento do Modelo ---")

# 1. Definir X (features) e y (target)
# Usamos a coluna 'texto_limpo' (do Bloco 4) e a coluna 'sentimento' (do Bloco 3)
print("[Passo 1/4] Definindo variáveis X e y...")
X = df['texto_limpo']
y = df['sentimento']

# 2. Dividir em treino e teste
# O stratify=y garante que a divisão mantenha a proporção de sentimentos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Total de amostras de treino: {len(X_train):,} amostras. Total de teste: {len(X_test):,} amostras.")

# 3. Construir o Pipeline do modelo
print("\n[Passos 3/4] Construindo o pipeline do modelo...")
# O Pipeline aplica primeiro o TF-IDF (vetorização) e depois a Regressão Logística (classificação)
modelo_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(max_iter=1000, random_state=42))
])

# 4. Treinar o modelo
print("\n[Passos 4/4] Iniciando o treinamento do modelo (isso leva alguns minutos)...")
modelo_pipeline.fit(X_train, y_train)
print("✅ Treinamento concluído!")

# 5. Fazer previsões no conjunto de teste e avaliar o modelo
y_pred = modelo_pipeline.predict(X_test)
acuracia = accuracy_score(y_test, y_pred)

print("\n--- PERFORMANCE DO MODELO ---")
print(f"Acurácia: {acuracia:.4f}")

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))


--- Iniciando a Construção e Treinamento do Modelo ---
[Passo 1/4] Definindo variáveis X e y...
Total de amostras de treino: 697,720 amostras. Total de teste: 174,431 amostras.

[Passos 3/4] Construindo o pipeline do modelo...

[Passos 4/4] Iniciando o treinamento do modelo (isso leva alguns minutos)...
✅ Treinamento concluído!

--- PERFORMANCE DO MODELO ---
Acurácia: 0.6534

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.56      0.16      0.24     62516
           1       0.66      0.93      0.78    111915

    accuracy                           0.65    174431
   macro avg       0.61      0.54      0.51    174431
weighted avg       0.63      0.65      0.59    174431



## 6. Salvando o Pipeline Final

Com o modelo treinado e avaliado, a etapa final é salvar o objeto `modelo_pipeline` em um arquivo. Isso nos permite reutilizar o modelo treinado no futuro (por exemplo, em nosso dashboard com Streamlit) sem a necessidade de retreiná-lo toda vez.

Usaremos a biblioteca `joblib` para salvar o modelo e, em seguida, a `google.colab.files` para baixá-lo para nosso computador.

In [9]:
## 6. Salvando o Pipeline Final

print("\n--- SALVAMENTO DO PIPELINE ---")

# Esta é a última correção. Garantimos que a variável CAMINHO_MODELO (que aponta para saved_models/) seja usada.
try:
    # 6.1 - Salva o Pipeline no Google Drive (na pasta saved_models/)
    joblib.dump(modelo_pipeline, CAMINHO_MODELO)
    print(f"✅ Pipeline do modelo salvo com sucesso em: {CAMINHO_MODELO}")

    # 6.2 - GERA O DOWNLOAD DIRETO (Recurso solicitado)
    from google.colab import files
    # Garante que o arquivo seja baixado com o nome correto
    files.download(CAMINHO_MODELO)
    print("\n⬇️ Download iniciado! Baixe o arquivo 'modelo_sentimento.joblib' para sua máquina.")

except Exception as e:
    print(f"ERRO ao salvar e baixar o modelo: {e}")

print("\nFim do Notebook. O novo modelo está pronto para ser usado no Streamlit!")


--- SALVAMENTO DO PIPELINE ---
✅ Pipeline do modelo salvo com sucesso em: /content/drive/MyDrive/Projeto_CampusParty_Sentimentos/saved_models/modelo_sentimento.joblib


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


⬇️ Download iniciado! Baixe o arquivo 'modelo_sentimento.joblib' para sua máquina.

Fim do Notebook. O novo modelo está pronto para ser usado no Streamlit!
